In [1]:
import urllib
import zipfile
import os
import numpy as np
import pandas
from pprint import pprint
from ftplib import FTP

pandas.set_option('display.max_colwidth', -1)

states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc']
states += ['de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn']
states += ['mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nm', 'nv', 'ny', 'oh', 'ok', 'or']
states += ['pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us']
states += ['ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

basedir = '/home/idies/workspace/persistent/censusdata/'
#local_dir = '/home/idies/workspace/raddick_acs_data/'
pumsdir = basedir + 'microdata/'

if not (os.getcwd() == basedir):
    os.chdir(basedir)
if not (os.path.exists(pumsdir)):
    os.makedirs(pumsdir)
os.chdir(pumsdir)
print('In directory: '+os.getcwd())

ftp = FTP('ftp2.census.gov')
ftp.login()
print(ftp.getwelcome())
#print(ftp.dir())
ftp.cwd('programs-surveys/acs/data/pums/2016/1-Year/')
print(ftp.nlst())
#thefilename = 'All_Geographies.zip'
#with open(thefilename, 'wb') as f:
#    ftp.retrbinary('RETR {0:}'.format(thefilename), f.write)
ftp.quit()
print('Done')

#dirurl = 'https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/'

#print('Writing zipfiles...')
#for onestate in states:
#    filename = 'csv_p{0:}.zip'.format(onestate)
#    remotefilename = dirurl + filename
#    localfilename = data_dir + filename
##    print('Reading {0:}'.format(remotefilename))
#    response = urllib.request.urlopen(remotefilename)
#    downloadedfile = response.read()
#    if (onestate in ('ak', 'fl', 'oh', 'tx', 'us', 'wy')):
#        print('Writing {0:}...'.format(filename))
#    with open(localfilename, 'wb') as f:
#        f.write(downloadedfile)
        

    
#    print('Reading file...')
#downloadedfile = response.read()
    
#print('Writing zipfile...')
#with open('All_Geographies.zip', 'wb') as f:
#    f.write(downloadedfile)

#print('Reading zipfile...')
#thezipfile = zipfile.ZipFile('All_Geographies.zip')

#print('Extracting files...')
#thezipfile.extractall()

#thezipfile.close()

#print('done')

In directory: /home/idies/workspace/persistent/censusdata/microdata
Traceback (most recent call last):
  File "/home/idies/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-cf8b5d767b14>", line 30, in <module>
    ftp = FTP('ftp2.census.gov')
  File "/home/idies/anaconda3/lib/python3.5/ftplib.py", line 118, in __init__
    self.connect(host)
  File "/home/idies/anaconda3/lib/python3.5/ftplib.py", line 156, in connect
    self.welcome = self.getresp()
  File "/home/idies/anaconda3/lib/python3.5/ftplib.py", line 235, in getresp
    resp = self.getmultiline()
  File "/home/idies/anaconda3/lib/python3.5/ftplib.py", line 221, in getmultiline
    line = self.getline()
  File "/home/idies/anaconda3/lib/python3.5/ftplib.py", line 203, in getline
    line = self.file.readline(self.maxline + 1)
  File "/home/idies/anaconda3/lib/python3.5/socket.py", line 576, in readinto
 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



KeyboardInterrupt: 

In [ ]:
print('Extracting files...')        
for onestate in states:
    thezipfilename = data_dir + 'csv_p{0:}.zip'.format(onestate)
    print(thezipfilename)
    thezipfile = zipfile.ZipFile(thezipfilename)    
    thezipfile.extractall()
    thezipfile.close()
    os.remove(thezipfilename)
print('Done')

In [ ]:
import numpy as np
import pandas
from pprint import pprint
import json
x_df = pandas.read_csv('microdata/ss16pri.csv', low_memory=False)
x_df = x_df.set_index('SERIALNO', drop=False)

# Capitalize names of all columns for consistency with metadata file
col_list = x_df.columns
new_col_list = []
for i in range(0, len(col_list)):
    new_col_list.append(col_list[i].upper())
x_df[new_col_list] = x_df[col_list]

x_df.sample(2)

In [ ]:
data_columns_list = x_df.sample(1).columns.sort_values().tolist()
for i in range(0, len(data_columns_list)):
    data_columns_list[i] = data_columns_list[i].upper()
data_columns_list.sort()
print('Columns in the person microdata:')
data_columns_list

In [ ]:
# PARSE METADATA
with open('microdata/pums_metadata_person.txt', 'rt', encoding='utf-8') as f:
    read_data = f.readlines()
f.close()

variables = {}
varnames = []
vardescription = []
newvarline = -1
counter = 0
for aline in read_data:
#    print(aline)
    maybevarname = aline.strip().split(' ')[0]
    
    if ((aline.isupper()) | ('UNEDITED' in aline)):                            # if the entire line is uppercase...
        thisvar = aline.strip().split('\t')             # it's a variable, whose name is the text up to the first tab...
        if not(thisvar[0].strip()[0].isnumeric()):      #  except that some all-caps lines are value codes starting with numbers
            newvarline = counter                        # ALSO, this is the start of a new variable group, so record its line number...
            thisvarname = thisvar[0].strip()
            variables[thisvarname] = {}
            variables[thisvarname]['name'] = thisvar[0].strip()
    if ((counter - newvarline) == 1):
        variables[thisvarname]['description'] = aline.strip()
    if ((counter - newvarline) >= 2):
        if (len(aline) > 0):
            valueslist = aline.strip().split(' .')
            if (len(valueslist) == 1):
                if (len(valueslist[0]) > 0):
                    variables[thisvarname]['note'] = aline.strip()
#                    print('note: ', valueslist[0])
            else:
#                print('...and then....', valueslist)
                variables[thisvarname]['values'] = {}
                for j in range(0,7):
                    variables[thisvarname]['values'][j] = j * 3

#        print('..........and then: ',aline)
    counter = counter + 1                            # increment the counter
    
metadata_columns_list = []
for (key, value) in variables.items():
    metadata_columns_list.append(key)
metadata_columns_list.sort()
print('Metadata about person microdata, parsed from pums_metadata_person.txt:')
print(metadata_columns_list)

In [ ]:
# Verify that the columns are the same in data and metadata

print('Columns in data list: {0:,.0f}'.format(len(data_columns_list)))
print('Columns in metadata list: {0:,.0f}'.format(len(metadata_columns_list)))

for z in metadata_columns_list:
    if (z not in data_columns_list):
        print(z)
        
for z in data_columns_list:
    if (z not in metadata_columns_list):
        print(z)

In [ ]:
# NEXT: How do we get the metadata to save in the same order?
better_search_order = np.empty((len(x_df.columns),2), dtype=object)
counter = 0
current_columns = x_df.columns.tolist()
for i in range(0, len(better_search_order)):
    better_search_order[i,0] = current_columns[i]
    better_search_order[i,1] = counter
    counter = counter + 1
better_search_order_df = pandas.DataFrame(better_search_order, columns={'column_name','sort_order'})

better_search_order_df = better_search_order_df.set_index('column_name')
better_search_order_df

In [ ]:
for (thiskey, thisvalue) in variables.items():
    variables[thiskey]['sort_order'] = better_search_order_df['sort_order'].loc[thiskey]
#pprint(variables)
variables_json = json.dumps(variables)
with open('/home/idies/workspace/raddick_acs_data/microdata_variables_person.json', 'w') as f:
    f.write(variables_json)
print('Done')

In [ ]:
x_df.to_csv('/home/idies/workspace/raddick_acs_data/microdata_person.csv')
print('Done')

In [ ]:
y_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/microdata_person.csv', index_col=0)
y_df = y_df.rename(columns={'SERIALNO.1': 'SERIALNO'})
#y_df.head(1)

with open('/home/idies/workspace/raddick_acs_data/microdata_variables_person.json', 'r') as f:
    z = f.read()
    
mdf_dict = json.loads(z)
pprint(mdf_dict)